Папка на GDrive:

https://drive.google.com/drive/folders/1FeUwrVfxp09xAifiMngwtfrRZ8nhtj8T?usp=sharing


In [3]:
from os import listdir

USE_GDRIVE = False


if USE_GDRIVE:
    from google.colab import drive
    drive.mount("/home/GDrive")
    DATA_PATH = f"/home/GDrive/MyDrive/made_2021_fraud_project/data"
else:
    DATA_PATH = "../../data"

GPS_PATH = "../../data/all_gps.csv"
ACCEL_PATH = "../../data/all_accel.csv"

listdir(DATA_PATH)

['unlabeled',
 'good_data',
 'all_gps.csv',
 'all_accel.csv',
 'drivers_with_gps_and_sl_problems',
 'rides_of_drivers.csv']

In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import folium
from haversine import haversine
from os import listdir


%matplotlib inline

In [5]:
def describe(df):
    display(pd.concat((df.dtypes, df.describe(datetime_is_numeric=True).T), axis=1))
    display(df.shape)

In [6]:
df_gps = pd.read_csv(GPS_PATH, parse_dates=[0, 3])
describe(df_gps)

,0,count,mean,min,25%,50%,75%,max,std
time,datetime64[ns],5050292,2021-04-06 10:23:00.788006400,2020-10-25 03:59:58,2021-01-16 06:40:41.750000128,2021-04-07 17:01:10.500000,2021-06-22 15:23:15.499999744,2021-08-28 02:47:58,NaN
lat,float64,5.05029e+06,55.743,42.98,55.6674,55.7496,55.8242,82.4343,0.171211
lon,float64,5.05029e+06,37.6021,26.1013,37.5009,37.5848,37.7032,135.46,0.401377
gps_time,datetime64[ns],5050292,2021-04-06 10:22:58.275877632,2020-10-25 03:59:56,2021-01-16 06:40:38.749999872,2021-04-07 17:01:10.500000,2021-06-22 15:23:15.499999744,2021-08-28 02:47:58,NaN
driver_hash,int64,5.05029e+06,3.03558e+17,-9.21858e+18,-4.06958e+18,1.02237e+17,4.79299e+18,9.20647e+18,5.27957e+18
fraud,float64,1.06611e+06,0.100867,0,0,0,0,1,0.301152


(5050292, 6)

In [7]:
df_accel = pd.read_csv(ACCEL_PATH, parse_dates=[0])
describe(df_accel)

,0,count,mean,min,25%,50%,75%,max,std
time,datetime64[ns],2172989,2020-11-22 15:27:12.714645504,2015-03-01 05:46:42.832594,2020-12-18 06:53:34.205120,2021-03-18 16:54:00.549220096,2021-06-12 11:02:18.664983040,2021-08-29 00:03:16.287344,NaN
x,float64,2.17299e+06,0.368934,-71.1054,-0.371101,0.184359,0.785298,65.1529,2.36914
y,float64,2.17299e+06,7.6569,-74.7916,7.63271,8.85376,9.50019,55.2294,3.4034
z,float64,2.17299e+06,3.83295,-65.5062,2.01402,3.69992,5.35365,76.4756,2.73072
lat,float64,2.17299e+06,55.7482,55.3163,55.6768,55.7512,55.8214,82.4343,0.16893
lon,float64,2.17299e+06,37.601,36.3143,37.5016,37.5863,37.686,108.856,0.376061
driver_hash,int64,2.17299e+06,-1.12657e+17,-9.21858e+18,-4.47912e+18,-4.14168e+17,4.43147e+18,9.20647e+18,5.3536e+18
fraud,float64,335525,0.315374,0,0,0,1,1,0.464666


(2172989, 8)

In [8]:
def map_driver(driver_gps, driver_accel):
    center = (driver_gps["lat"].mean(), driver_gps["lon"].mean())
    color = driver_accel.head(1)["fraud"].replace({1:"red", 0:"green", np.nan:"yellow"}).values[0]
    m = folium.Map(location=center, zoom_start=10, legend=True)
    folium.PolyLine(driver_gps[["lat", "lon"]], color='blue', weight=4, label="tracks", opacity=0.8).add_to(m)
    folium.PolyLine(driver_accel[["lat", "lon"]], color=color, weight=2, label="accel", opacity=0.8).add_to(m)
    return m

In [9]:
FORCE_FRAUD = None

if not FORCE_FRAUD is None:
    cur_driver = np.random.choice(df_gps.loc[df_gps["fraud"] == FORCE_FRAUD, "driver_hash"].unique())
else:
    cur_driver = np.random.choice(df_gps["driver_hash"].unique())

# пример водителя в большим количесвтом сматченных точек
# cur_driver = -6646726793274239750
cur_driver

-4880039172951741036

In [10]:
driver_gps = df_gps[df_gps["driver_hash"] == cur_driver]
driver_accel = df_accel[df_accel["driver_hash"] == cur_driver]

display(
    f"Current Driver:{cur_driver}",
    driver_gps.describe(datetime_is_numeric=True).T,
    driver_accel.describe(datetime_is_numeric=True).T
)

map_driver(driver_gps, driver_accel)

'Current Driver:-4880039172951741036'

,count,mean,min,25%,50%,75%,max,std
time,4963,2021-04-06 06:25:57.121700608,2021-04-05 10:56:32,2021-04-06 05:48:41.500000,2021-04-06 06:50:44,2021-04-06 08:45:30,2021-04-06 17:54:30,NaN
lat,4963,55.8705,55.8622,55.8694,55.8695,55.8715,55.8834,0.00332114
lon,4963,37.6782,37.6487,37.6786,37.6789,37.6791,37.6987,0.00670867
gps_time,4963,2021-04-06 06:25:57.106387200,2021-04-05 10:56:31,2021-04-06 05:48:41.500000,2021-04-06 06:50:44,2021-04-06 08:45:30,2021-04-06 17:54:29,NaN
driver_hash,4963,-4.88004e+18,-4.88004e+18,-4.88004e+18,-4.88004e+18,-4.88004e+18,-4.88004e+18,1024.1
fraud,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,count,mean,min,25%,50%,75%,max,std
time,1284,2021-04-06 09:58:54.561161216,2021-04-04 07:56:29.681801,2021-04-05 17:53:04.995990272,2021-04-06 17:47:16.320483584,2021-04-07 05:47:41.089060352,2021-04-07 17:53:27.875349,NaN
x,1284,-0.551203,-4.58667,-0.895737,-0.62236,-0.291252,6.3222,0.938937
y,1284,1.33652,-4.93486,-0.19706,0.289276,1.87362,10.5024,2.74457
z,1284,9.30166,0.915924,9.08173,9.80226,9.99069,12.8071,1.43602
lat,1284,55.8721,55.8622,55.8693,55.8716,55.8748,55.8872,0.00459053
lon,1284,37.6793,37.6487,37.6699,37.6802,37.6879,37.7098,0.0124449
driver_hash,1284,-4.88004e+18,-4.88004e+18,-4.88004e+18,-4.88004e+18,-4.88004e+18,-4.88004e+18,1024.4
fraud,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_merged = pd.merge_asof(driver_gps, driver_accel,
              left_on="gps_time",
              right_on="time",
              direction="nearest",
              tolerance=pd.Timedelta("30s"),
              suffixes=("", "_r"),
              allow_exact_matches=False)

df_matched = df_merged.dropna(thresh=7).copy() \
    .drop(columns=["driver_hash_r", "fraud_r"]) \
    .rename(columns={"time_r": "accel_time", "lat_r": "accel_lat", "lon_r": "accel_lon"})

display(f"Matched: {df_matched.shape[0]} of {df_merged.shape[0]}")
describe(df_matched)

'Matched: 626 of 4963'

,0,count,mean,min,25%,50%,75%,max,std
time,datetime64[ns],626,2021-04-06 05:22:22.091054336,2021-04-05 17:45:37,2021-04-05 17:53:25.750000128,2021-04-06 09:08:11.500000,2021-04-06 17:46:03.249999872,2021-04-06 17:54:30,NaN
lat,float64,626,55.8698,55.8624,55.8682,55.87,55.8718,55.8752,0.00309738
lon,float64,626,37.6757,37.6487,37.6653,37.678,37.6844,37.6965,0.0115066
gps_time,datetime64[ns],626,2021-04-06 05:22:22.052715776,2021-04-05 17:45:37,2021-04-05 17:53:25.750000128,2021-04-06 09:08:11.500000,2021-04-06 17:46:03.249999872,2021-04-06 17:54:29,NaN
driver_hash,int64,626,-4.88004e+18,-4.88004e+18,-4.88004e+18,-4.88004e+18,-4.88004e+18,-4.88004e+18,1024.82
fraud,float64,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
accel_time,datetime64[ns],626,2021-04-06 05:22:22.459925760,2021-04-05 17:46:05.170200,2021-04-05 17:53:05.009872896,2021-04-06 09:08:21.930461952,2021-04-06 17:46:16.405174016,2021-04-06 17:54:29.234848,NaN
x,float64,626,-0.491331,-4.2457,-0.965134,-0.636124,-0.237717,3.07634,0.947638
y,float64,626,2.24828,-4.91331,-0.0570831,0.927567,5.17123,8.40274,3.14246
z,float64,626,8.95196,4.79947,8.31094,9.6557,9.90575,11.345,1.49971


(626, 12)

In [13]:
cols_gps = ["driver_hash", "fraud", "time", "lat", "lon", "gps_time"]
df_matched_gps = df_matched[cols_gps]

cols_accel = ["driver_hash", "fraud", "accel_time", "accel_lat", "accel_lon", "x", "y", "z"]
df_matched_accel = df_matched[cols_accel] \
    .rename(columns={c:c.replace("accel_", "") for c in cols_accel})

map_driver(df_matched_gps, df_matched_accel)

In [14]:
gps_time_diff = df_matched["gps_time"].diff()
gps_time_diff.fillna(gps_time_diff.mean(), inplace=True)
display(gps_time_diff.round("5min").value_counts())

accel_time_diff = df_matched["accel_time"].diff()
accel_time_diff.fillna(accel_time_diff.diff().mean(), inplace=True)
display(accel_time_diff.round("5min").value_counts())

0 days 00:00:00    624
0 days 08:30:00      1
0 days 15:05:00      1
Name: gps_time, dtype: int64

0 days 00:00:00    624
0 days 08:30:00      1
0 days 15:05:00      1
Name: accel_time, dtype: int64

In [16]:
df_matched["track"] = gps_time_diff.round("7min").cumsum().dt.seconds
number_of_tracks = len(df_matched["track"].unique())
display(f"Number of tracks: {number_of_tracks}")

df_matched = df_matched[["driver_hash", "fraud", "track"] + cols_gps[2:] + cols_accel[2:]].copy()
describe(df_matched)

'Number of tracks: 3'

,0,count,mean,min,25%,50%,75%,max,std
driver_hash,int64,626,-4.88004e+18,-4.88004e+18,-4.88004e+18,-4.88004e+18,-4.88004e+18,-4.88004e+18,1024.82
fraud,float64,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
track,int64,626,40736.6,0,0,54180,84840,84840,34967.7
time,datetime64[ns],626,2021-04-06 05:22:22.091054336,2021-04-05 17:45:37,2021-04-05 17:53:25.750000128,2021-04-06 09:08:11.500000,2021-04-06 17:46:03.249999872,2021-04-06 17:54:30,NaN
lat,float64,626,55.8698,55.8624,55.8682,55.87,55.8718,55.8752,0.00309738
lon,float64,626,37.6757,37.6487,37.6653,37.678,37.6844,37.6965,0.0115066
gps_time,datetime64[ns],626,2021-04-06 05:22:22.052715776,2021-04-05 17:45:37,2021-04-05 17:53:25.750000128,2021-04-06 09:08:11.500000,2021-04-06 17:46:03.249999872,2021-04-06 17:54:29,NaN
accel_time,datetime64[ns],626,2021-04-06 05:22:22.459925760,2021-04-05 17:46:05.170200,2021-04-05 17:53:05.009872896,2021-04-06 09:08:21.930461952,2021-04-06 17:46:16.405174016,2021-04-06 17:54:29.234848,NaN
accel_lat,float64,626,55.8698,55.864,55.8682,55.87,55.8719,55.875,0.00309013
accel_lon,float64,626,37.6756,37.6497,37.6662,37.6776,37.6844,37.6951,0.0116498


(626, 13)

In [17]:
df_matched

,driver_hash,fraud,track,time,lat,lon,gps_time,accel_time,accel_lat,accel_lon,x,y,z
479,-4880039172951741036,NaN,0,2021-04-05 17:45:37,55.863960,37.666228,2021-04-05 17:45:37,2021-04-05 17:46:05.170200,55.863962,37.666295,-0.961548,6.581802,7.114517
480,-4880039172951741036,NaN,0,2021-04-05 17:45:41,55.863962,37.666295,2021-04-05 17:45:41,2021-04-05 17:46:05.170200,55.863962,37.666295,-0.961548,6.581802,7.114517
481,-4880039172951741036,NaN,0,2021-04-05 17:45:43,55.863965,37.666293,2021-04-05 17:45:43,2021-04-05 17:46:05.170200,55.863962,37.666295,-0.961548,6.581802,7.114517
482,-4880039172951741036,NaN,0,2021-04-05 17:45:46,55.863967,37.666284,2021-04-05 17:45:46,2021-04-05 17:46:05.170200,55.863962,37.666295,-0.961548,6.581802,7.114517
483,-4880039172951741036,NaN,0,2021-04-05 17:45:49,55.863970,37.666298,2021-04-05 17:45:49,2021-04-05 17:46:05.170200,55.863962,37.666295,-0.961548,6.581802,7.114517
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958,-4880039172951741036,NaN,84840,2021-04-06 17:54:04,55.862361,37.681453,2021-04-06 17:54:04,2021-04-06 17:54:29.234848,55.864396,37.683559,0.200165,1.617889,10.541046
4959,-4880039172951741036,NaN,84840,2021-04-06 17:54:07,55.862354,37.681454,2021-04-06 17:54:07,2021-04-06 17:54:29.234848,55.864396,37.683559,0.200165,1.617889,10.541046
4960,-4880039172951741036,NaN,84840,2021-04-06 17:54:27,55.862354,37.681454,2021-04-06 17:54:27,2021-04-06 17:54:29.234848,55.864396,37.683559,0.200165,1.617889,10.541046
4961,-4880039172951741036,NaN,84840,2021-04-06 17:54:30,55.863160,37.682063,2021-04-06 17:54:29,2021-04-06 17:54:29.234848,55.864396,37.683559,0.200165,1.617889,10.541046


In [18]:
center = (df_matched["lat"].mean(), df_matched["lon"].mean())
m = folium.Map(location=center, zoom_start=10, legend=True)

color = driver_accel.head(1)["fraud"].replace({1:"red", 0:"green", np.nan:"yellow"}).values[0]

for track in df_matched["track"].unique():
    fg = folium.FeatureGroup(name=f"track{track}")
    gps_track = df_matched[df_matched["track"] == track]
    folium.PolyLine(gps_track[["lat", "lon"]], color="blue", weight=4, label="accel", opacity=0.8).add_to(fg)
    accel_track = df_matched[df_matched["track"] == track]
    folium.PolyLine(accel_track[["accel_lat", "accel_lon"]], color='red', weight=2, label="tracks", opacity=0.8).add_to(fg)
    fg.add_to(m)

folium.map.LayerControl('topleft', collapsed= False).add_to(m)
m

### Вывод - такая разбивка на поездки выглядит вполне правдоподобной